#### **Entrez, BLAST y Alineamiento Múltiple de Secuencias**
###### **Alliance meeting Paraguay EUSAT-RCS consortium, 2024**

###### *Diseñado por Jorge G. García, 2024*
---
Bienvenidos a una breve introducción en genómica comparativa. Las consultas a bases de datos especializadas y los alineamientos de secuencias constituyen la columna vertebral de innumerables proyectos de investigación en todo el mundo, y la lucha contra la tuberculosis (TB) hace uso extensivo de estas herramientas. Algunos de los usos de estas tecnologías incluyen:

Identificar mutaciones puntuales en genes seleccionados a través de diferentes cepas de Mycobacterium tuberculosis (Mtb).
Rastrear la historia natural de Mtb a través de regiones y períodos de tiempo.
Revelar patrones de propagación y transmisión mediante análisis filogenético.
Descubrir potenciales objetivos para nuevos fármacos comparando secuencias genéticas entre cepas resistentes a fármacos y susceptibles.
Evaluar la diversidad genética de las cepas de Mtb para informar el desarrollo de vacunas y personalizar tratamientos para variantes específicas.
A medida que nos embarquemos en este viaje, exploraremos cómo recuperar secuencias de ADN de bases de datos especializadas de manera programática y cómo compararlas de maneras perspicaces. Este taller está diseñado para equiparte con el conocimiento y las herramientas para contribuir al esfuerzo global contra la TB a través del lente de la bioinformática. Aunque utilizaremos datos reales y actualizados durante la mayor parte del ejercicio, está dirigido a una amplia audiencia y no requiere ningún conocimiento previo sobre pipelines de bioinformática, ¡ni siquiera de Python!

Si no tienes experiencia alguna con la programación, ¡no te preocupes! Este ejercicio está diseñado para llevarte sin esfuerzo directamente al final. Si por alguna razón lo rompes, reinicia y voilà. Si ya estás familiarizado con la programación, este ejercicio debería ser útil para entender cómo se recuperan y alinean los datos biológicos. ¡Siéntete libre de jugar con otros parámetros y explorar cómo funciona el código!

Por último, pero no menos importante, a lo largo del ejercicio verás muchos hipervínculos esparcidos por las secciones. Haz clic en ellos para acceder a recursos adicionales que enriquecerán tu experiencia, agregarán contexto o revelarán hechos interesantes sobre la lección.

Con todo esto en mente, toma una respiración profunda y presiona el botón Ejecutar en la primera celda. Bienvenido a la bioinformática.


In [ ]:
from Bio import Entrez, SeqIO, Phylo, AlignIO, Align
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import pandas as pd
import numpy as np
import time

¡Felicidades! Acabas de importar las bibliotecas necesarias para realizar todos los análisis en este ejercicio.

En programación, una biblioteca es una colección de rutinas precompiladas - conjuntos de instrucciones - que un programa puede usar. Estas rutinas están diseñadas para realizar tareas específicas, como manejar archivos y datos, realizar cálculos matemáticos o gestionar conexiones de red - y eso es exactamente lo que vamos a hacer con ellas. Muchas de estas bibliotecas que usaremos, como pandas para la gestión de datos tabulares o el poderoso numpy para operaciones matemáticas, se utilizan en todo el mundo en muchos campos diferentes, desde finanzas hasta la predicción del tiempo.

También verás que muchos de nuestros comandos de importación comienzan con "Bio" - abreviatura de [Biopython](https://biopython.org/), una biblioteca especializada y completa para realizar operaciones con datos biológicos.

Pero antes de proceder, deberíamos tener muy claro lo que estamos buscando. El genoma de [Mycobacterium tuberculosis](https://www.microbiologyresearch.org/content/journal/micro/10.1099/mic.0.000601) tiene aproximadamente 4.4 millones de bases de longitud, codificando alrededor de 4000 genes e innumerables regiones no codificantes - pero aún importantes. Como investigadores, a menudo estamos interesados en segmentos específicos de este todo, particularmente aquellos asociados a rasgos en el patógeno, y uno de los más notables es la resistencia a los antibióticos.

Hay amplia evidencia de la asociación entre la activación de ciertos genes en Mtb y la resistencia a los antibióticos. Algunos de los más destacados son:
- [**rpoB**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8502021/): Las mutaciones en el gen *rpoB*, que codifica la subunidad β de la RNA polimerasa, son responsables de la resistencia a la rifampicina, un medicamento clave de primera línea para el tratamiento de la TB. La resistencia a la rifampicina es un marcador crítico para la TB multidrogorresistente (MDR-TB).

- [**katG**](https://journals.asm.org/doi/10.1128/jb.175.13.4255-4259.1993): Las mutaciones en este gen, que codifica catalasa-peroxidasa, están asociadas con la resistencia a la isoniazida. La isoniazida, otro fármaco anti-TB de primera línea, requiere activación por KatG para ejercer su efecto antibacteriano. Las mutaciones en katG pueden llevar a una pérdida de esta activación, resultando en resistencia.

- [**inhA**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7856099/#:~:text=KatG%20and%20inhA%20gene%20mutations%20are%20the%20main,are%20activated%20by%20monooxygenase%20EthA%20and%20catalase-peroxidase%20KatG.): Este gen codifica una enzima involucrada en la síntesis del ácido micólico, un componente de la pared celular de Mtb. Las mutaciones en la región promotora de *inhA* también pueden conferir resistencia a la isoniazida.

- [**embB**](https://pubmed.ncbi.nlm.nih.gov/33210572/#:~:text=Ethambutol%20%28EMB%29%20is%20one%20of%20the%20first-line%20drugs,for%20around%2070%25%20clinical%20EMB%20resistant%20M.%20tuberculosis.): El gen *embB* está asociado con la resistencia al etambutol, un fármaco de primera línea que tiene como objetivo la síntesis de la pared celular. Las mutaciones en embB pueden afectar la capacidad del fármaco para inhibir la síntesis de arabinogalactano, lo que lleva a la resistencia.

Vamos a investigar el primero, *rpoB*, y su papel en el desarrollo de resistencia a la rifampicina. [Rifampicina](https://go.drugbank.com/drugs/DB01045) es un antibiótico ampliamente utilizado en el tratamiento de la TB, y funciona inhibiendo la enzima RNA polimerasa, que es crucial para la síntesis de RNA. Cuando se usa en terapia combinada, la rifampicina ayuda a prevenir el desarrollo de cepas de TB resistentes a los medicamentos. Sin embargo, la monoterapia o el uso indebido de la rifampicina pueden llevar a la aparición de resistencia, lo cual es una preocupación creciente en todo el mundo.

Ahora realizarás un simulacro de búsqueda en la base de datos Entrez, buscando cualquier publicación que contenga secuencias nucleotídicas (ADN en nuestro caso) y las palabras "Mycobacterium", "tuberculosis" y "rifampicina" en su título. ¡Presiona el botón Ejecutar en la siguiente celda y ve qué sucede!



In [ ]:
def fetch_and_filter_sequences():
    seq_length = 200
    print('Obteniendo secuencias...')
    search_handle = Entrez.esearch(db="nucleotide", term="Mycobacterium tuberculosis rifampicin[Title]", retmax=100)
    search_results = Entrez.read(search_handle)
    search_handle.close()
    id_list = search_results['IdList']

    fetch_handle = Entrez.efetch(db="nucleotide", id=id_list, rettype="fasta", retmode="text")
    sequences = list(SeqIO.parse(fetch_handle, "fasta"))
    fetch_handle.close()

    filtered_sequences = [seq for seq in sequences if len(seq.seq) <= seq_length]
    SeqIO.write(filtered_sequences, "filtered_sequences.fasta", "fasta")
    print(f"Filtered {len(filtered_sequences)} sequences at most {seq_length}bp.")

def fetch_and_filter_sequences_sim():
    seq_length = 200
    print('Obteniendo secuencias...')
    with open('filtered_sequences.fasta', 'r') as file:
        filtered_sequences = list(SeqIO.parse(file, 'fasta'))
        time.sleep(5)
        print(f"{len(filtered_sequences)} secuencias filtradas con un máximo de {seq_length} pares de bases.")


fetch_and_filter_sequences_sim()

Como puedes ver, la celda anterior contiene dos funciones, llamadas casi exactamente igual. Bueno, no hay misterio en ello, la primera es la real, y la segunda un simple simulacro que emite los mismos mensajes. ¿Por qué estamos haciendo esto? ¿No se supone que debemos recuperar datos reales en tiempo real? Bueno, hay dos razones para esto:

Esta operación se ejecuta en segundo plano y, dependiendo de la complejidad de la consulta, puede tomar varios minutos mirando un terminal vacío antes de que termine.
Toma recursos computacionales de la base de datos Entrez para realizar una consulta. Podemos ahorrarles este esfuerzo redundante teniendo los archivos ya preparados y listos para usar, ¡justo como haces tú!
Sin embargo, eres muy bienvenido a verificar el archivo exacto que se habría descargado de otra manera, llamado "filtered_sequences.fasta". Un archivo FASTA es un documento de texto plano que generalmente contiene un encabezado y una secuencia dada. Por ejemplo, nuestro archivo FASTA contiene dos secuencias, siendo la primera:

```
>LN651305.1 Mycobacterium tuberculosis partial rpoB gene, isolate Kw9101-13, rifampicin resistance determining region
GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGCCAATTCATGTACCAG
AACAACCCGCTGTCGGGGTTGACCCACAAGCGCCGACTGTCGGCGCTGGGGCCCGGCGGT
CTGTCACGTGAGCGTGCCGGGCTG
```


Las entradas de archivos FASTA siempre comienzan con un encabezado. La mayor parte de la estructura de este encabezado más allá del primer símbolo ">" no es un requisito estricto de los archivos FASTA - podríamos cambiarlo y nuestro script funcionaría igual de bien - pero contiene información útil para nosotros:

1. ID de secuencia, un identificador exclusivo de la secuencia.
2. El organismo de origen.
3. La ubicación de la secuencia.
4. El rol de la secuencia en el genoma.

Sin embargo, los archivos FASTA son ampliamente utilizados no por sus encabezados, sino por sus secuencias. Son una forma ligera y directa de compartir secuencias, sean proteínas o ácidos nucleicos. La biblioteca Biopython contiene rutinas para analizar y extraer información estructurada de estos archivos de una manera conveniente para su posterior exploración. Presiona el botón Ejecutar en la siguiente celda para continuar.

In [ ]:
def seq_reader():
    with open('filtered_sequences.fasta', 'r') as file:
        sequences = list(SeqIO.parse(file, 'fasta'))
        print(f"{len(sequences)} secuencias leídas.")
        return sequences
    
seq_reader()

Mucho mejor! Podemos ver toda la información recuperada por categorías. Pero entonces uno se pregunta, ¿de qué base de datos acabo de obtener estos datos? ¿Por qué sus descripciones son tan similares? ¿Debería confiar en ella?

¡Buenas preguntas! Vamos a explorar por unos segundos el mundo de las bases de datos biológicas y el lugar que Entrez ocupa entre ellas.

El sistema de bases de datos [Entrez](https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html) es un conjunto integrado y completo de bases de datos desarrollado y mantenido por el [Centro Nacional de Información Biotecnológica (NCBI)](https://www.ncbi.nlm.nih.gov/). Proporciona acceso a una amplia gama de datos biológicos, incluidas secuencias de nucleótidos y proteínas, datos genómicos, compuestos químicos, referencias de literatura y mucho más. El sistema Entrez está diseñado para facilitar la búsqueda y recuperación de información biológica a través de varias bases de datos, convirtiéndolo en un recurso crucial para investigadores en los campos de biología molecular, genética, medicina y bioinformática.

TL;DR: Piénsalo como una especie de Google para datos bioquímicos. Su naturaleza integrada permite una exploración fluida de datos relacionados a través de diferentes aspectos biológicos, haciéndolo un recurso esencial para investigadores alrededor del mundo.

Nuestra función simplemente "pidió" a la base de datos de Nucleótidos, parte de Entrez, por cualquier entrada que coincidiera con nuestros criterios, y devolvió una colección de secuencias. Luego, aquellas secuencias con una longitud igual o menor a 200 pares de bases fueron seleccionadas para continuar el análisis aguas abajo. Ni la base de datos ni nuestra función saben lo que planeamos hacer con estas secuencias, así que depende de nosotros analizarlas, comenzando por verificar si son idénticas o no. Presiona el botón Ejecutar en la siguiente celda para continuar.

In [ ]:
def seq_identical():
    sequence_info = seq_reader()
    sequence_holder = list(np.zeros(len(sequence_info), dtype=int))
    for i, seq in enumerate(sequence_info):
        sequence_holder[i] = seq.seq

    all_identical = set(tuple(seq) for seq in sequence_holder) == 1
    
    if all_identical:
        print('Todas las secuencias son idénticas.')
    else:
        print('Las secuencias no son idénticas.')

seq_identical()

Resulta que... ¡no lo son! Esta es solo una manera simple y rápida de verificar si un número indeterminado de secuencias son exactamente iguales - en contenido y en orden - sin tener que revisar cada posible comparación par a par.

**Desafío extra para amantes de Python** -> Crea y ejecuta otra función que cumpla el mismo objetivo <u>sin realizar comparaciones par a par</u>.

Podemos continuar comparando estas secuencias. De hecho, vamos a aprovechar que son sólo dos. ¡Es hora de introducir el alineamiento de secuencias par a par!

Imagina las siguientes dos secuencias:
- *Secuencia A*: `ACTGTCGCA`
- *Secuencia B*: `ACCGTGGCA`

¿Puedes notar las diferencias? ¿Serías capaz de alinear ambas secuencias de tal manera que podamos identificar fácilmente las coincidencias y las diferencias? Algo como lo que obtienes si presionas el siguiente botón Ejecutar:


In [ ]:
def test_aligner():
    seq_a = 'ACTGTCGCA'
    seq_b = 'ACCGTGGCA'
    aligner = Align.PairwiseAligner()

    aligner.mode = 'global'     
    aligner.match_score = 1      
    aligner.mismatch_score = 1     
    aligner.open_gap_score = 0.1
    aligner.extend_gap_score = 0.1      

    alignments = aligner.align(seq_a, seq_b)
    print(f'Alineamiento de las secuencias:\n')
    print(alignments[0])

test_aligner()

Estoy seguro de que sí, pero las pipelines bioinformáticas tratan rutinariamente con secuencias de miles de bases de longitud. ¿Cómo podemos alinear matemáticamente dos secuencias así de una manera consistente y reproducible? Aquí entra en juego la matriz de alineamiento:



|   | - | A | C | T | G | T | C | G | C | A  |
|---|---|---|---|---|---|---|---|---|---|---|
| - | 0 |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| T |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |   |

Crear una matriz de puntuación de alineación es un paso fundamental para entender los algoritmos de alineación de secuencias. La matriz ayuda a visualizar el proceso de alinear dos secuencias al puntuar similitudes, diferencias y huecos. Para esta explicación, nos centraremos en un esquema de puntuación simple: +1 por coincidencia, -1 por diferencia y -2 por hueco. El objetivo es alinear estas secuencias para maximizar la puntuación de alineación basada en nuestro esquema de puntuación. Nuestro primer valor (*i0*, *j0*) es un 0, representando el costo de desalinear dos secuencias vacías.

Comenzaremos inicializando nuestra primera fila con penalizaciones por hueco - el costo de saltar una base para alinear mejor las secuencias. Este paso es necesario para comenzar a llenar la matriz.


|   | - | A | C | T | G | T | C | G | C | A  |
|---|---|---|---|---|---|---|---|---|---|---|
| - | 0 | -2| -4| -6| -8| -10| -12| -14| -16| -18|
| A |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| T |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |   |

Luego, las reglas del juego son simples. Comenzando desde la primera posición:
1. Si los caracteres coinciden, suma la puntuación de coincidencia (+1) al valor diagonal superior izquierdo.
2. Si los caracteres no coinciden, toma el máximo de:
    - El valor diagonal más la puntuación por no coincidencia (-1).
    - El valor izquierdo más la penalización por hueco (-2).
    - El valor superior más la penalización por hueco (-2).
3. Repite el paso 2 para cada celda en la matriz hasta el último valor (*i<sub>m</sub>*, *j<sub>n</sub>*).

Deberías obtener una tabla como esta (no te preocupes, no tienes que hacerlo manualmente)



|   | - | A  | C  | T  | G  | T  | C  | G  | C  | A  |
|---|---|----|----|----|----|----|----|----|----|----|
| - | 0 | -2 | -4 | -6 | -8 | -10| -12| -14| -16| -18|
| A | -2| 1  | -1 | -3 | -5 | -7 | -9 | -11| -13| -15|
| C | -4| -1 | 2  | 0  | -2 | -4 | -6 | -8 | -10| -12|
| C | -6| -3 | 0  | 1  | -1 | -1 | -3 | -5 | -7 | -9 |
| G | -8| -5 | -2 | -1 | 2  | 0  | 0  | -2 | -4 | -6 |
| T |-10| -7 | -4 | -3 | 0  | 3  | 1  | -1 | -3 | -5 |
| G |-12| -9 | -6 | -3 | -2 | 1  | 2  | 0  | 0  | -2 |
| G |-14|-11 | -8 | -5 | -2 | -1 | 2  | 3  | 1  | -1 |
| C |-16|-13 | -10| -7 | -4 | -3 | 0  | 1  | 4  | 2  |
| A |-18|-15 | -12| -9 | -6 | -5 | -2 | -1 | 2  | 5  |


Y ahora que esto ha terminado, necesitamos **reconstruir** nuestro camino basado en las elecciones que hicimos (diagonal, izquierda o arriba) en un proceso llamado *trazado inverso*. Primero debemos elegir la celda correcta para comenzar este *trazado inverso*:

- Para el alineamiento global ([Needleman-Wunsch](https://www.wikiwand.com/en/Needleman%E2%80%93Wunsch_algorithm)), comienza desde la celda inferior derecha de la matriz.
- Para el alineamiento local ([Smith-Waterman](https://www.wikiwand.com/en/Smith%E2%80%93Waterman_algorithm)), comienza desde la celda con la puntuación más alta en toda la matriz.

Pasos de *Trazado Inverso*:
- Si el camino óptimo se mueve diagonalmente (de (*i*, *j*) a (*i-1*, *j-1*)), indica una coincidencia o no coincidencia. Añade los caracteres correspondientes de ambas secuencias al alineamiento y muévete a la celda (*i-1*, *j-1*).
- Si el camino óptimo se mueve hacia arriba (de (*i*, *j*) a (*i-1*, *j*)), indica un hueco en la segunda secuencia. Añade un carácter de hueco (-) al alineamiento para la segunda secuencia, el carácter de la primera secuencia y muévete a la celda (*i-1*, *j*).
- Si el camino óptimo se mueve hacia la izquierda (de (*i*, *j*) a (*i*, *j-1*)), indica un hueco en la primera secuencia. Añade un carácter de hueco (-) al alineamiento para la primera secuencia, el carácter de la segunda secuencia y muévete de la celda (*i*, *j*) a (*i*, *j-1*).

Repite el proceso de verificar movimientos diagonales, hacia arriba o hacia la izquierda hasta que alcances la celda de finalización:
- Para el alineamiento global, esta es la celda superior izquierda (*i0*, *j0*).
- Para el alineamiento local, es cualquier celda en el borde donde comenzó el trazado inverso (dependiendo de las reglas del algoritmo, podrías detenerte una vez que alcances una puntuación de 0 para incluir solo la región local de alta puntuación).

Podemos generar una implementación programática del algoritmo global con unas pocas líneas de código. Presiona el botón Ejecutar para ver cómo se desempeña.


In [ ]:
def needleman_wunsch_test():
    # Initialize the scoring matrix
    seq1 = 'ACTGTCGCA'
    seq2 = 'ACCGTGGCA'
    match_score = 1
    mismatch_penalty = -1
    gap_penalty = -2
    m, n = len(seq1), len(seq2)
    score_matrix = [[0 for _ in range(n+1)] for _ in range(m+1)]

    # Initialize the edges with gap penalties
    for i in range(m + 1):
        score_matrix[i][0] = i * gap_penalty
    for j in range(n + 1):
        score_matrix[0][j] = j * gap_penalty

    # Fill in the scoring matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

    print(pd.DataFrame(score_matrix, columns = ['-'] + list(seq1), index = ['-'] + list(seq2)))

needleman_wunsch_test()


¡Uf! ¡Qué dolor! Afortunadamente, la biblioteca Biopython contiene módulos que están aquí para aliviarnos de este esfuerzo. Siéntate, toma un respiro y deja que el programa lo haga en un segundo presionando el botón Ejecutar. Te lo has ganado.

In [ ]:
def entrez_aligner():
    sequence_info = seq_reader()
    print('Alineando secuencias...')
    aligner = Align.PairwiseAligner()

    aligner.mode = 'global'     # Choose "global" or "local"
    aligner.match_score = 1      # Choose your match score here, for example -3
    aligner.mismatch_score = 1     # Choose your mismatch score here, for example 3
    aligner.open_gap_score = 0.1     # Choose your gap penalties here, for example 0.6
    aligner.extend_gap_score = 0.1      # Choose your gap penalties here, for example 0.6

    alignments = aligner.align(sequence_info[0].seq, sequence_info[1].seq)
    print(alignments[0])

entrez_aligner()

Ahora esa es una manera mucho mejor de visualizar un alineamiento. La línea media se compone de tres símbolos:
- "|" representa una coincidencia.
- "-" representa un hueco.
- "." representa una sustitución.

Siéntete libre de experimentar con diferentes parámetros de puntuación y modos de alineamiento. Para referencia:
- El modo global se centra en maximizar el alineamiento de extremo a extremo de ambas secuencias.
- El modo local se centra en maximizar el alineamiento en las regiones más similares de ambas secuencias.

Observa cómo la modificación de las diferentes puntuaciones altera el resultado del alineamiento. Como resumen rápido, recuerda los siguientes puntos:
1. Utilizamos algoritmos de alineamiento par a par para obtener la mejor puntuación de coincidencia entre nuestras secuencias dadas nuestro esquema de puntuación.
2. Llenamos la matriz secuencialmente basándonos en un conjunto de instrucciones.
3. Luego construimos el alineamiento *retrotrayendo* nuestros pasos. Dependiendo de nuestras elecciones escribimos bases o huecos hasta que nuestro alineamiento esté completo.

*Por cierto, imagina hacer los pasos anteriores manualmente con secuencias de cientos o incluso [miles](https://www.wikiwand.com/en/Masochism_(disambiguation)) de bases.*

Ahora que hemos superado eso, aprenderemos cómo realizar un [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi). BLAST (Herramienta de Búsqueda de Alineamiento Local Básico) es un algoritmo fundamental para comparar una secuencia biológica individual (como aminoácidos para proteínas o nucleótidos para ADN/ARN) con una biblioteca o base de datos de secuencias, y funciona con los mismos principios exactos de alineamiento que acabas de aprender en el paso anterior.

BLAST funciona de la siguiente manera:
1. Secuencia de Consulta: El usuario envía una secuencia a BLAST, que sirve como la consulta.
2. Búsqueda en la Base de Datos: BLAST compara la secuencia de consulta contra una base de datos de secuencias para encontrar coincidencias.
3. Identificar Coincidencias: Identifica pares de secuencias de alta puntuación (HSPs) buscando coincidencias de palabras entre la consulta y las secuencias de la base de datos y luego extendiendo estas coincidencias para encontrar alineamientos más largos sin huecos significativos.
4. Puntuación: Cada alineamiento potencial se puntúa basado en el número de coincidencias, no coincidencias y huecos, con puntuaciones más altas indicando mejores alineamientos.
5. Resultados: BLAST devuelve una lista de secuencias de la base de datos que son similares a la secuencia de consulta, incluyendo alineamientos y medidas estadísticas de significancia (como los [valores E](https://www.wikiwand.com/en/E-values), que estiman el número de coincidencias por azar en una base de datos de un tamaño dado).

Para un refresco visual y agradable sobre alineamientos y BLAST, consulta este muy buen artículo cortesía de [Nature](https://www.nature.com/scitable/topicpage/basic-local-alignment-search-tool-blast-29096/).

Para continuar con el ejercicio, elegiremos una de las dos secuencias al azar y la BLASTearemos. ¡Basta de charla! Pulsa ese botón Ejecutar y deja que la computadora maneje el resto.

In [ ]:
def blast_sequence_individual():
    sequences = list(SeqIO.parse("filtered_sequences.fasta", "fasta"))

    print(f'BLASTing secuencia {sequences[0].id}...')
    result_handle = NCBIWWW.qblast("blastn", 
                                    "nt", 
                                    sequences[0].seq,
                                    expect=0.001,
                                    hitlist_size=100)
                                    
    with open(f"blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()
    print(f'Successfully BLASTed sequence {sequences[0].id}.')

def blast_sequence_individual_sim():
    sequences = list(SeqIO.parse("filtered_sequences.fasta", "fasta"))
    print(f"BLASTing sequence {sequences[0].id}...")
    time.sleep(5)
    print(f'BLAST ejecutado con éxito en secuencia {sequences[0].id}.')
            

blast_sequence_individual_sim()

Y otro simulacro más. Las razones para hacerlo son prácticamente las mismas que antes. Imagina tener que calcular todas esas horribles matrices de alineamiento, pero contra millones de secuencias, buscando las que devuelven la mejor probabilidad estadística de una coincidencia no aleatoria. ¡Hablando de encontrar una aguja en un pajar genético! De hecho, el [genio](https://blastalgorithm.com/) de la búsqueda en la base de datos BLAST es la velocidad y eficiencia con la que el algoritmo maneja tal cacería colosal. Sin embargo, si ejecutas la función no simulada, espera esperar unos largos minutos.

Nuestro BLAST ha sido ejecutado. Siéntete libre de explorar el archivo llamado "blast_result.xml" haciendo clic en él. Verás que hay un patrón que se repite dentro del documento, y lo que parecen ser métricas y secuencias. Estos son los resultados devueltos desde la base de datos, y ahora tenemos que analizarlos de manera ordenada. Presiona el botón Ejecutar para olvidarte de los cómo hacerlo.

In [ ]:
def parse_blast_results_to_fasta():
    hsps = []
    with open(f'blast_result.xml') as result_file:
        blast_record = NCBIXML.read(result_file)
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                hsps.append(hsp)
    
    print(f'{len(hsps)} HSPs decodificados. Verifique el archivo "aggregated_sequences.fasta"')
    print(f'Primer HSP: {hsps[0]}.')

parse_blast_results_to_fasta()

¡Cien HSPs! Eso significa que de nuestra consulta original, el BLAST ha encontrado cien secuencias coincidentes y ha calculado las métricas que nos permiten medir cuantitativamente el grado de similitud y longitud entre ellas.

Cien secuencias. ¿Sería posible alinear **todas** ellas? ¿Sería necesario realizar todos los posibles alineamientos par a par entre todos los posibles pares de secuencias? Eso sería nada menos que 100<sup>2</sup> = 10000 alineamientos par a par. Y no tendrían en cuenta un puntaje de alineación combinado, sino solo para pares individuales. Para esto, necesitamos una nueva herramienta matemática: Alineamiento Múltiple de Secuencias - no te preocupes, no entraremos en los detalles matemáticos complicados. Presiona el botón Ejecutar para realizar otro simulacro.


In [ ]:
def clustal_msa_aligner():
    print('Analizando secuencias con ClustalOmega...')
    clustalomega_cline = ClustalOmegaCommandline(infile="aggregated_sequences.fasta", outfile="aligned_sequences.fasta", verbose=True, auto=True, force=True)
    stdout, stderr = clustalomega_cline()
    print(f'Alineamiento Múltiple de Secuencias completado.Verifique el archivo "aligned_sequences.fasta"')

def clustal_msa_aligner_sim():
    print('Alineando secuencias con ClustalOmega...')
    time.sleep(5)
    print(f'Alineamiento Múltiple de Secuencias completado. Verifique el archivo "aligned_sequences.fasta"')

clustal_msa_aligner_sim()


En este punto, podrías estar pensando que la mitad de este código no funciona en un entorno real. Demasiados simulacros. Bueno, en este caso es porque estamos ejecutando una herramienta externa al entorno. Si el alineamiento par a par parece lo suficientemente tedioso desde el punto de vista matemático, es Disney World en comparación con la complejidad de realizar un MSA, y se puede verificar fácilmente por la cantidad de tiempo y recursos que toma realizar uno simple entre una cantidad moderada de secuencias no muy largas. Sin embargo, si quieres tener una segunda opinión, sigue estas instrucciones:

1. Haz clic en el siguiente [enlace](https://www.ebi.ac.uk/jdispatcher/msa/clustalo).
2. Selecciona "DNA".
3. Haz clic en "Choose File" y selecciona "aggregated_sequences.fasta".
4. Escribe un título para el trabajo.
5. Haz clic en enviar.
6. Siéntate y disfruta mientras un clúster de computadoras de alto rendimiento al otro lado del Atlántico procesa los números.

O

1. Consulta el siguiente [enlace](https://www.ebi.ac.uk/Tools/services/rest/mview/result/mview-I20240318-212727-0482-15810022-p1m/aln-html) para una solución precalculada en HTML.

Sin importar cómo elijas hacerlo, el resultado será un archivo de salida que ya tienes disponible: "aligned_sequences.fasta". Será el último ingrediente que necesitamos para lo que nos espera. ¡Presiona ese botón Ejecutar, estamos casi terminados!

In [ ]:
def distance_matrix_calculator():
    print('Calculando Matriz de Distancias...')
    align = AlignIO.read("aligned_sequences.fasta", "fasta")
    calculator = DistanceCalculator('identity') # Try other options like "blastn", "trans", "benner6", "benner22", "benner74"
    dm = calculator.get_distance(align)

    # Convert the identity matrix to a pandas DataFrame for nicer output
    split_names = [_.split("|")[-1] for _ in dm.names]
    df = pd.DataFrame(dm.matrix, columns=split_names, index=split_names)
    print('Muestra de Matriz de Identidad:')
    print(df.iloc[:7, :7])

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
distance_matrix_calculator()

Nuestros algoritmos han generado una [matriz de distancias](https://www.wikiwand.com/en/Distance_matrix). Esta es una matriz cuadrada, diagonal que contiene las "distancias" entre todos los pares de nuestras secuencias ya alineadas. La diagonal está compuesta de 0s, porque no hay "distancia" entre una secuencia y ella misma. Siéntete libre de experimentar con las opciones en la función y ver cómo cambian los puntajes en la matriz de salida de muestra. Cuando estés listo, presiona el botón Ejecutar una última vez.

In [ ]:
def phylogeny_calculator():
    print('Construyendo Árbol Filogenético...')
    align = AlignIO.read("aligned_sequences.fasta", "fasta")
    calculator = DistanceCalculator('identity') # Try other options like "blastn", "trans", "benner6", "benner22", "benner74"
    dm = calculator.get_distance(align)
    constructor = DistanceTreeConstructor(calculator, 'nj')  # Neighbor-Joining method. Try "upgma" (Unweighted Pair Group Method with Arithmetic Mean)
    tree = constructor.build_tree(align)
    Phylo.write(tree, "phylogeny.xml", "phyloxml")

    # Optionally, display the tree
    Phylo.draw_ascii(tree)
    
phylogeny_calculator()

Por último, pero no menos importante, aquí está el prometido [árbol filogenético](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7123334/). Nuestras funciones nos han llevado a un punto en el que podemos comparar todas las secuencias con precisión matemática y dibujar tal árbol. La distribución de las ramas nos habla de los posibles ancestros comunes de cada [variante genómica](https://www.genome.gov/about-genomics/educational-resources/fact-sheets/human-genomic-variation), mientras que la longitud de las ramas representa la distancia figurativa entre las secuencias. Siéntete libre de modificar nuevamente las opciones en la función constructora y ver cómo cambian la forma del árbol.

Este árbol nos informa sobre la proximidad de las secuencias analizadas, y es un buen punto de partida para agrupar nuestras variantes de una manera informativa.

Felicidades por haber llegado al final de la lección. Estas son algunas de las herramientas básicas que los bioinformáticos de todo el mundo utilizan rutinariamente en la lucha contra la TB; pero como dice el dicho, el agujero del conejo es mucho, mucho más profundo. Grandes esfuerzos de investigación internacionales están haciendo uso de estas tecnologías, así como tratando de mejorarlas. Como cierre de este ejercicio, te invito a echar un vistazo a algunos de los proyectos actuales más fascinantes que involucran tecnologías como estas a diario:
- [El Proyecto BioGenoma de la Tierra (EBP)](https://www.earthbiogenome.org/), tiene como objetivo secuenciar, catalogar y caracterizar los genomas de toda la biodiversidad eucariota de la Tierra en un período de 10 años.
- [El Proyecto ENCODE](https://www.encodeproject.org/) tiene como objetivo anotar el genoma humano completo. Es decir, no solo identificar la secuencia de ADN y sus variaciones, sino qué hace cada parte de la secuencia. [El Proyecto 1000 Genomas](https://www.internationalgenome.org/home) sigue siendo uno de los mayores repositorios de datos de variación genómica humana, y es muy utilizado por la comunidad biomédica. Ambos son sucesores espirituales del [Proyecto Genoma Humano](https://www.genome.gov/human-genome-project), a menudo considerado el Proyecto Manhattan de la biología.
- [AGAThA (Aceleración por GPU del Alineamiento de Secuencias Guiado para el Mapeo de Lecturas Largas)](https://arxiv.org/abs/2403.06478) tiene como objetivo llevar el alineamiento de secuencias al siguiente nivel permitiendo arquitecturas paralelas GPU en algoritmos de alineamiento, mientras que [Embed-Search-Align usando Modelos Transformer](https://ar5iv.labs.arxiv.org/html/2309.11087) toma inspiración en las similitudes entre secuencias de ADN y lenguajes humanos, utilizando tecnología originalmente desarrollada para el procesamiento del lenguaje natural (NLP).

Si deseas profundizar más en la ciencia y el arte de la gestión y análisis de datos biológicos, recomiendo los siguientes recursos (no estoy afiliado de ninguna manera con ellos):
- Biología Molecular de la Célula, de Alberts et al., el manual de referencia renombrado para todo lo molecular, ya en su 7ª edición.
- [El Manual de Biostars](https://www.biostarhandbook.com/), un enfoque amigable para principiantes a todo lo relacionado con bioinformática, desde scripts hasta Secuenciación de Nueva Generación.
- [De la Línea Celular a la Línea de Comando](https://divingintogeneticsandgenomics.ck.page/), para aquellos biólogos que quieren dar el salto al lado computacional de las cosas.

¡Espero que te haya gustado este pequeño ejercicio, y que puedas llevar algo de él de vuelta a tu vida profesional y cotidiana! Por favor, no dudes en hacer cualquier pregunta, ¡y espero escuchar de todos ustedes pronto!